# Lambda = 4, lambda = 0.2: EvolvedOperatorAnsatz

In [1]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import time

import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import EvolvedOperatorAnsatz

import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

# Load H

In [3]:
data_coeff, data_str = open_n_process("../utility/pauliH_L4_g0.2.txt")
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.000897
Max absolute value is 1.15
Mean absolute value is 0.03431
None
E_exact = 3.13406


In [4]:
#get the indices of the N largest elements
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)
ind25, ops25, vals25 = get_k_largest_ops(data_coeff, data_str, 25)
ind30, ops30, vals30 = get_k_largest_ops(data_coeff, data_str, 30)
ind40, ops40, vals40 = get_k_largest_ops(data_coeff, data_str, 40)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_Hp25 =[Pauli(f'{ops25[i]}') for i in range(len(ops25))]
ops_Hp30 =[Pauli(f'{ops30[i]}') for i in range(len(ops30))]
ops_Hp40 =[Pauli(f'{ops40[i]}') for i in range(len(ops40))]

In [5]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_Hp25 = EvolvedOperatorAnsatz(ops_Hp25, reps=1, insert_barriers=True)
ev_op_Hp30 = EvolvedOperatorAnsatz(ops_Hp30, reps=1, insert_barriers=True)
ev_op_Hp40 = EvolvedOperatorAnsatz(ops_Hp40, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_Hp25_2f = EvolvedOperatorAnsatz(ops_Hp25, reps=2, insert_barriers=True)
ev_op_Hp30_2f = EvolvedOperatorAnsatz(ops_Hp30, reps=2, insert_barriers=True)
ev_op_Hp40_2f = EvolvedOperatorAnsatz(ops_Hp40, reps=2, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_Hp25, ev_op_Hp30, ev_op_Hp40, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_Hp25_2f, ev_op_Hp30_2f, ev_op_Hp40_2f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_Hp25', 'ev_op_Hp30', 'ev_op_H40',
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_Hp25_2f', 'ev_op_Hp30_2f', 'ev_op_Hp40_2f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 25, 30, 40, 30, 40, 50, 60, 80]


In [5]:
#additional ansatze
ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ansatz_list = [ev_op_Hp15_3f, ev_op_Hp20_3f]
ansatz_names =['ev_op_Hp15_3f', 'ev_op_Hp20_3f']
print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [45, 60]


In [11]:
#ev_op_Hp15.decompose().draw('mpl')
#ev_op_Hp20.decompose().draw('mpl')
#ev_op_Hp25.decompose().draw('mpl')
#ev_op_Hp30.decompose().draw('mpl')

# VQE individual runs

In [6]:
seed = 170
iterations = 650
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA

In [7]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 3.12106
Length of this optimization 199, time taken = 208.496 

At step 1, with ev_op_Hp20
VQE result: 3.11756
Length of this optimization 270, time taken = 375.408 

At step 2, with ev_op_Hp25
VQE result: 3.17782
Length of this optimization 306, time taken = 433.771 

At step 3, with ev_op_Hp30
VQE result: 3.32965
Length of this optimization 325, time taken = 500.911 

At step 4, with ev_op_H40
VQE result: 7.35099
Length of this optimization 527, time taken = 1269.803 

At step 5, with ev_op_Hp15_2f
VQE result: 3.13053
Length of this optimization 320, time taken = 459.036 

At step 6, with ev_op_Hp20_2f
VQE result: 3.23598
Length of this optimization 526, time taken = 990.898 

At step 7, with ev_op_Hp25_2f
VQE result: 3.33135
Length of this optimization 631, time taken = 1313.553 

At step 8, with ev_op_Hp30_2f
VQE result: 3.46483
Length of this optimization 650, time taken = 1467.775 

At step 9, with ev_op_Hp40_2f
VQE result: 4.33957
Length of

In [9]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv('l4_l02_op_ev_20_30_40_indv_cobyla.csv')

In [7]:
# Additional ansatze
r_cobyla2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15_3f
VQE result: 3.13999
Length of this optimization 440, time taken = 687.047 

At step 1, with ev_op_Hp20_3f
VQE result: 3.23875
Length of this optimization 642, time taken = 1832.641 



In [8]:
df1b = pd.concat([r_cobyla2[i] for i in range(len(r_cobyla2))], axis = 1)
df1b.to_csv('l4_l02_op_ev_a_20_30_40_indv_cobyla.csv')
df1b

,ev_op_Hp15_3f,ev_op_Hp20_3f
0,3.782361,15.211639
1,4.993923,15.390675
2,5.602874,14.819995
3,5.101522,12.998426
4,3.782361,14.853416
...,...,...
637,NaN,3.232475
638,NaN,3.232820
639,NaN,3.245695
640,NaN,3.232097


# SPSA

In [8]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 3.19377
Length of this optimization 1351, time taken = 1861.069 

At step 1, with ev_op_Hp20
VQE result: 3.15612
Length of this optimization 1351, time taken = 2061.173 

At step 2, with ev_op_Hp25
VQE result: 3.14337
Length of this optimization 1351, time taken = 2153.691 

At step 3, with ev_op_Hp30
VQE result: 3.16268
Length of this optimization 1351, time taken = 26373.709 

At step 4, with ev_op_H40
VQE result: 3.17260
Length of this optimization 1351, time taken = 2971.854 

At step 5, with ev_op_Hp15_2f
VQE result: 3.12846
Length of this optimization 1351, time taken = 2173.352 

At step 6, with ev_op_Hp20_2f
VQE result: 3.19462
Length of this optimization 1351, time taken = 2589.496 

At step 7, with ev_op_Hp25_2f
VQE result: 3.12774
Length of this optimization 1351, time taken = 2730.965 

At step 8, with ev_op_Hp30_2f
VQE result: 3.13160
Length of this optimization 1351, time taken = 3023.525 

At step 9, with ev_op_Hp40_2f
VQE result: 9

In [11]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv('l4_l02_op_ev_20_30_40_indv_spsa.csv')

In [9]:
r_spsa2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15_3f
VQE result: 3.18062
Length of this optimization 1351, time taken = 3119.072 

At step 1, with ev_op_Hp20_3f
VQE result: 3.11734
Length of this optimization 1351, time taken = 4001.807 



In [10]:
df2b = pd.concat([r_spsa2[i] for i in range(len(r_spsa2))], axis = 1)
df2b.to_csv('l4_l02_op_ev_a_20_30_40_indv_spsa.csv')